In [1]:
import os
import logging
import sys
import nest_asyncio
import json

In [2]:
cd ..

/home/tsunn/Workspace/iai-lab/sosci/codes/Graph-RAG


In [3]:
from dotenv import load_dotenv
from llama_index.llms.openai import OpenAI
from llama_index.core import (
    Settings,
    Document, get_response_synthesizer,
    PropertyGraphIndex,
    StorageContext
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.graph_stores.neo4j import Neo4jGraphStore

from core.data.processing import process_jsonl_data
from embeddings.LocalEmbedding import LocalEmbedding
from llm.TogetherLLM import TogetherLLM
from llm.Mistral import Mistral

/home/tsunn/anaconda3/envs/nlm_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO:datasets:PyTorch version 2.5.1 available.


In [4]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("llama_index").setLevel(logging.INFO)
nest_asyncio.apply()

In [5]:
data_path = "core/data/annotations.jsonl"
llm = Mistral()
splitter = SentenceSplitter()
embedder = LocalEmbedding()

Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 17772.47it/s]


Mistral LLM initialized.
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: Alibaba-NLP/gte-large-en-v1.5
Loaded model: Alibaba-NLP/gte-large-en-v1.5


In [6]:
try:
    chunks = process_jsonl_data(data_path, mode="document")
except Exception as e:
    print(f"Error processing data: {e}")
    sys.exit(1)

Loaded 74669 documents


Parsing nodes: 100%|██████████| 74669/74669 [00:13<00:00, 5616.07it/s]


Finish splitting 77319 chunks


Processing chunks: 100%|██████████| 77319/77319 [00:00<00:00, 248253.98it/s]

Processed 77319 chunks


In [7]:
_chunks = chunks[:10]
len(_chunks)

10

In [8]:
_chunks = [Document.from_dict(chunk) for chunk in _chunks]
tmp = []
for chunk in _chunks:
    temp = splitter._get_splits_by_fns(chunk.text)[0]
    _tmp = []
    res = ""
    for t in temp:
        if len(res) > 32:
            _tmp.append(Document(text=res))
            res = ""
        res += t
    if len(res) > 0:
        _tmp.append(Document(text=res))
    
    # Do batch embedding
    embs = embedder.get_text_embedding_batch(
        texts = [doc.text for doc in _tmp],
        show_progress = True
    )
    for i, doc in enumerate(_tmp):
        doc.embedding = embs[i]
    tmp.extend(_tmp)

_chunks = tmp

len(_chunks)

Generating embeddings: 100%|██████████| 5/5 [00:26<00:00,  5.37s/it]


119

In [9]:
_chunks

[Document(id_='8f5913f1-569d-4cba-9453-1b94f4a2ee82', embedding=[-0.060546875, -0.498046875, -0.6953125, 0.474609375, 0.396484375, -0.12451171875, -0.64453125, 0.6875, -0.53515625, -0.59765625, 1.0, 0.047607421875, -0.58984375, -0.064453125, -1.0078125, 0.134765625, -0.65625, 0.1865234375, -0.359375, -0.796875, -0.1796875, -0.625, 0.80859375, -0.41796875, 0.66015625, -0.205078125, -0.1435546875, 0.67578125, -0.298828125, 0.88671875, 0.33984375, -1.09375, 1.734375, 0.2216796875, 0.1611328125, 0.640625, 0.2275390625, 1.8125, -0.609375, 0.9453125, -0.91015625, -0.302734375, 0.42578125, -0.0751953125, 0.12060546875, 0.146484375, -0.1806640625, -1.0859375, -0.73046875, 0.16796875, -2.140625, 0.8046875, -0.2578125, 0.58984375, 0.48046875, 0.2080078125, -0.64453125, -0.357421875, -1.1875, 0.109375, -0.58984375, 0.5, -0.423828125, -0.890625, -1.3125, -0.1298828125, -0.474609375, -0.255859375, -1.4296875, 0.53125, -0.78125, 0.5078125, -0.2578125, -1.140625, 1.5703125, -0.91796875, -0.37109375, 

In [10]:
graph_store = Neo4jGraphStore(
    username=os.environ["NEO4J_USERNAME"],
    password=os.environ["NEO4J_PASSWORD"],
    url=os.environ["NEO4J_URI"]
)

INFO:neo4j.notifications:Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE CONSTRAINT IF NOT EXISTS FOR (e:Entity) REQUIRE (e.id) IS UNIQUE` has no effect.} {description: `CONSTRAINT constraint_1ed05907 FOR (e:Entity) REQUIRE (e.id) IS UNIQUE` already exists.} {position: None} for query: '\n                CREATE CONSTRAINT IF NOT EXISTS FOR (n:Entity) REQUIRE n.id IS UNIQUE;\n                '


In [11]:
storage_context = StorageContext.from_defaults(graph_store=graph_store)

In [12]:
index = PropertyGraphIndex.from_documents(
    documents=[Document.from_dict(chunk) for chunk in _chunks],
    llm=llm,
    embed_model=embedder,
    storage_context=storage_context,
    include_embeddings=False,
    max_triplets_per_chunk=2,
    show_progress=True
)

Extracting paths from text:   0%|          | 0/119 [00:00<?, ?it/s]

In [13]:
query_engine = index.as_query_engine(
    llm = llm,
    include_text=False, response_mode="tree_summarize"
)

In [ ]:
response = query_engine.query("Tell me more about Interleaf")

In [ ]:
response

In [ ]:
query_engine = index.as_query_engine(
    llm = llm,
    # embed_model = embedder,
    include_text=False,
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=5
)

In [ ]:
response = query_engine.query(
    "Effort"
)

In [ ]:
response

In [ ]:
nodes = response.source_nodes

# Iterate through the nodes to access their embeddings and calculate the length
for i, node in enumerate(nodes):
    if node.node.embedding is not None:
        
        embedding_length = len(node.node.embedding)
        print(f"Node {i + 1}: Embedding Length = {embedding_length}")
    else:
        print(f"Node {i + 1}: Embedding is None")